**LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [1]:
#@ GETTING LIBRARIES AND DEPENDENCIES: UNCOMMENT BELOW: 
# !python /content/simplepreprocessor.py

In [2]:
#@ IMPORTING LIBRARIES AND PACKAGES: 
import cv2
import numpy as np
import os

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from simplepreprocessor import SimplePreprocessor
from simplepreprocessor import SimpleDatasetLoader 
from imutils import paths

**IMAGE PREPROCESSOR:**
- I will build an image preprocessor that resizes the image, ignoring the aspect ratio. 

In [3]:
#@ INITIALIZING SIMPLE PREPROCESSOR:
class SimplePreprocessor:                                       # Defining Simple Preprocessor. 
    def __init__(self, width, height, inter=cv2.INTER_AREA):    # Initializing Constructor Function. 
        self.width = width                                      # Initialization. 
        self.height = height                                    # Initialization. 
        self.inter = inter                                      # Initialization. 

    def preprocess(self, image):                                # Defining Preprocess Function. 
        return cv2.resize(image, (self.width, self.height), 
                          interpolation=self.inter)             # Resizing Image. 

**IMAGE LOADER:**

In [4]:
#@ INITIALIZING DATASET LOADER: 
class SimpleDatasetLoader:                                      # Defining Simple Image Loader. 
    def __init__(self, preprocessors=None):                     # Initializing Constructor Function. 
        self.preprocessors = preprocessors                      # Initialization. 
        if self.preprocessors is None:                          # Inspection. 
            self.preprocessors = []                             # Initializing Empty List.
    
    def load(self, imagePaths, verbose=-1):                     # Defining Load Function. 
        data, labels = [], []                                   # Initializing Empty List.
        for (i, imagePath) in enumerate(imagePaths):
            image = cv2.imread(imagePath)                       # Reading Image. 
            label = imagePath.split(os.path.sep)[-2]            # Getting Labels. 
            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)                 # Preprocessing Image. 
            data.append(image)                                  # Updating Data. 
            labels.append(label)                                # Updating Label. 
            if verbose > 0 and i > 0 and (i+1)%verbose == 0:
                print("[INFO] processed {}/{}".format(
                    i + 1, len(imagePaths)))                    # Showing Updates. 
        return (np.array(data), np.array(labels))               # Initializing Array of Data. 

**K-NEAREST NEIGHBOR:**

In [6]:
#@ GETTING DATASET: UNCOMMENT BELOW: 
# !unzip /content/drive/MyDrive/Data/animals.zip

In [8]:
#@ PREPROCESSING DATASET: 
imagePaths = list(paths.list_images("/content/animals"))         # Getting Image Paths. 
sp = SimplePreprocessor(32, 32)                                  # Initializing Image Resizing. 
sdl = SimpleDatasetLoader(preprocessors=[sp])                    # Initializing Dataset Loader. 
(data, labels) = sdl.load(imagePaths, verbose=500)               # Initializing Data and Labels. 
data = data.reshape((data.shape[0], 3072))                       # Initializing Flatten. 

#@ ENCODING LABELS: 
le = LabelEncoder()                                              # Initializing Label Encoder. 
labels = le.fit_transform(labels)                                # Encoding Labels as Integers. 
(trainX, testX, trainY, testY) = train_test_split(
    data, labels, test_size=0.25, random_state=42)               # Initializing Training and Testing. 

#@ K-NEAREST NEIGHBOR ALGORITHMS: 
model = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)           # Initializing KNN Classifier. 
model.fit(trainX, trainY)                                        # Training KNN Classifier. 
print(classification_report(testY, model.predict(testX), 
                            target_names=le.classes_))           # Inspecting Classification Report. 

[INFO] processed 500/3000
[INFO] processed 1000/3000
[INFO] processed 1500/3000
[INFO] processed 2000/3000
[INFO] processed 2500/3000
[INFO] processed 3000/3000
              precision    recall  f1-score   support

        cats       0.41      0.53      0.46       249
        dogs       0.37      0.49      0.42       239
       panda       0.80      0.33      0.46       262

    accuracy                           0.45       750
   macro avg       0.52      0.45      0.45       750
weighted avg       0.53      0.45      0.45       750

